## Vamos a dar comienzo a la parte de modelado

In [37]:
import pyarrow as pa
import pyarrow.parquet as pq

import scipy as sp
import re

from sklearn.metrics.pairwise import cosine_similarity
import operator

from math import sqrt
from numpy import mean, var
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy.stats import skew




In [25]:
df_modelado = pd.read_parquet('df_modelado.parquet')

df_modelado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51967 entries, 0 to 58398
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    51967 non-null  object
 1   eval       51967 non-null  int64 
 2   item_name  51967 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


Vamos a proceder a ordenar esta Dataframe transponiendo y creando matrices 

In [26]:
matriz = pd.pivot_table(df_modelado,values='eval',index='user_id',columns='item_name',fill_value=0)

matriz

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ace--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ionex--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2SV-vuLB-Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Azsael-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zvanik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zynxgameth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Quise hacer de esta manera la normalización pero a la hora de transponer se me desmadraba todo

Normalización Z-Score por filas
matriz = matriz.sub(matriz.mean(axis=1), axis=0).div(matriz.std(axis=1), axis=0)
print("\nNormalización Z-Score por filas:")
matriz


In [27]:
# Definición de la función de normalización
def normalizar(fila):
    mean_val = np.mean(fila)
    min_val = np.min(fila)
    max_val = np.max(fila)
    normalized_row = (fila - mean_val) / (max_val - min_val)
    return normalized_row

# Aplicación de la función de normalización a cada fila de la matriz usuario-ítem
matriz_normalizada = matriz.apply(normalizar, axis=1)

matriz_normalizada

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301
--ace--,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301
--ionex--,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,...,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603
-2SV-vuLB-Kg,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,...,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904
-Azsael-,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,...,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958
zvanik,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,...,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904
zynxgameth,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301


In [28]:
matriz_normalizada_tr = matriz_normalizada.T

matriz_normalizada_tr

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-GM-Dragon,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zynxgameth,zyr0n1c,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
0RBITALIS,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
"10,000,000",-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
100% Orange Juice,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
1001 Spikes,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
the static speaks my name,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
theBlu,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301


Ahora vamos a crear una matriz de similitud mediante coseno (formula que nos proporcionaron para poder crearlo esto)

In [39]:
similitud = cosine_similarity(matriz_normalizada_tr)

In [40]:
df_similitud = pd.DataFrame(similitud, index = matriz_normalizada_tr.index, columns = matriz_normalizada_tr.index)

In [41]:
df_similitud 

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,1.000000,0.021942,0.011078,-0.000682,0.020927,0.006752,0.010435,0.019619,0.019910,0.006572,...,0.012973,0.010176,0.012260,0.006811,0.005543,0.004451,0.216437,0.021507,-0.011369,0.000690
0RBITALIS,0.021942,1.000000,0.005958,-0.000182,0.011108,0.003709,0.005601,0.010445,0.010593,0.003549,...,0.006930,0.005447,0.006535,0.003691,0.003025,0.002476,-0.000484,0.011403,-0.005703,0.000494
"10,000,000",0.011078,0.005958,1.000000,-0.000823,0.005602,0.001364,0.002572,0.005143,0.005245,0.001545,...,0.003319,0.002563,0.003185,0.001557,0.001189,0.000789,-0.000945,0.005805,-0.004221,-0.000267
100% Orange Juice,-0.000682,-0.000182,-0.000823,1.000000,-0.000359,-0.001140,-0.000690,-0.000588,-0.000537,-0.000608,...,-0.000578,-0.000546,-0.000433,-0.000732,-0.000775,-0.001005,-0.001398,-0.000258,-0.002526,-0.001056
1001 Spikes,0.020927,0.011108,0.005602,-0.000359,1.000000,0.003408,0.005277,0.009928,0.010076,0.003322,...,0.006563,0.005147,0.006204,0.003442,0.002799,0.002244,-0.000639,0.010887,-0.005780,0.000340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.004451,0.002476,0.000789,-0.001005,0.002244,0.000079,0.000797,0.001946,0.002012,0.000393,...,0.001167,0.000857,0.001175,0.000343,0.000166,1.000000,-0.001026,0.002377,-0.002933,-0.000583
the static speaks my name,0.216437,-0.000484,-0.000945,-0.001398,-0.000639,-0.001187,-0.000808,-0.000840,-0.000795,-0.000675,...,-0.000737,-0.000667,-0.000588,-0.000798,-0.000821,-0.001026,1.000000,-0.000550,-0.002260,-0.001021
theBlu,0.021507,0.011403,0.005805,-0.000258,0.010887,0.003580,0.005462,0.010224,0.010371,0.003452,...,0.006773,0.005318,0.006393,0.003584,0.002928,0.002377,-0.000550,1.000000,-0.005736,0.000428


In [42]:
df_similitud = df_similitud.to_parquet('df_similitud.parquet',engine='pyarrow')